In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv("dataset/9_class/training.csv")
df.dropna(inplace=True, axis=0)

label = {
    'Asam Lambung': 0,
    'Batuk': 1, 
    'Cedera': 2, 
    'Demam': 3,
    'Gatal': 4, 
    'Gigi': 5, 
    'Jerawat': 6, 
    'Menstruasi': 7, 
    'Sakit Kepala': 8
}

df['kategori'] = df['kategori'].map(label)

In [3]:
df.head()

,tanya,jawab,kategori
0,umur saya masih 18 tahun saya tidak obesitasti...,"\n\n<p>Alo, terima kasih atas pertanyaannya un...",7
1,hai saya berusia 16 tahun 3 bulan saya mempuny...,"\n\n<p>Selamat pagi, terima kasih telah bertan...",5
2,jika saat saya melakukan sit up kenapa perut s...,"<p>Dear Azizi,\n\nSakit perut sebelah kiri bag...",2
3,belakangan ini saya mengalami diare tapi susah...,"\n\n<p>Alo Adi,</p>\n<p>Terima kasih atas pert...",3
4,ter saya perempuan berusia 12 tahun sampe saat...,"\n\n<p>Alo, terimakasih atas pertanyaannya.</p...",7


In [4]:
from helper import load_model_ml
import tensorflow as tf
import joblib

model = tf.keras.models.load_model("model/model_3_87_85.h5")
vector = joblib.load(f"notebook/tokenizer.pkl")

In [9]:
input_users = "tubuh saya menggigil sejak tadi malam dok, suhu tubuh juga naikk. Apakah saya demam? cara cepat mengatasinya bagaimana yaa?"

# Sequence
sequence_testing = vector.texts_to_sequences([input_users])

# Padding
max_sequence_length = 200
padded_testing = tf.keras.preprocessing.sequence.pad_sequences(sequence_testing, 
                                                                maxlen=max_sequence_length, 
                                                                padding='post', 
                                                                truncating='post')

In [10]:
prediction = model.predict(padded_testing).argmax()
filter_data = df[df['kategori']==prediction]

sequence_data = vector.texts_to_sequences(filter_data['tanya'])
padded_data = tf.keras.preprocessing.sequence.pad_sequences(sequence_data, 
                                                                maxlen=max_sequence_length, 
                                                                padding='post', 
                                                                truncating='post')

cosine_similarities = cosine_similarity(padded_testing, padded_data)
most_similar_doc_index = cosine_similarities.argmax()

most_similar_doc = filter_data.iloc[most_similar_doc_index]
print(most_similar_doc["jawab"])

1/1 [==============================] - 0s 44ms/step




<p>Halo, Eka P. Handayani. Terima kasih atas pertanyaan yang disampaikan ke Alodokter. Kami memahami kekhawatiran yang Anda rasakan.</p>
<p>Pada anak usia 3 bulan, imunisasi yang dianjurkan sudah diberikan meliputi hepatitis B ke-1-3, polio oral dan injeksi 1-2. BCG, DTP ke-1-2, Hib ke-1-2, PCV ke-1, dan rotavirus ke-1.</p>
<p>Demam merupakan salah satu gejala umum yang dapat terjadi pascaimunisasi. Sekitar 42,9% dari keseluruhan kejadian ikutan pasca imunisasi adalah demam. Tidak menutup kemungkinan bahwa demam yang terjadi pascaimunisasi dapat menimbulkan kejang demam, namun umumnya kejadian ini sangat jarang dan hanya terjadi pada anak dengan faktor risiko kejang demam sebelumnya.</p>
<p>Jangan ragu untuk tetap melakukan imunisasi sesuai dengan jadwal yang dianjurkan. Meski ada kemungkinan risiko efek samping seperti demam, manfaat yang diperoleh dari imunisasi jauh lebih besar bagi kesehatan anak dan masyarakat secara umum. Dokter pun dapat memberikan obat penurun panas untuk ana

In [6]:
model = load_model_ml("model/model_1p1_88_85.pkl")
vector = joblib.load(f"notebook/count_vectorizer.pkl")

input_users = ["saya sakit kepala ampe mau putus"]

input_transform = vector.transform(input_users)  # Perhatikan bahwa input_users harus dibungkus dalam list
prediction = model.predict(input_transform)

filter_data = df[df['kategori']==prediction[0]]

tf_matrix = vector.transform(filter_data['tanya'])

cosine_similarities = cosine_similarity(input_transform, tf_matrix)
most_similar_doc_index = cosine_similarities.argmax()

most_similar_doc = filter_data.iloc[most_similar_doc_index]

# print(most_similar_doc['jawab'])
most_similar_doc_index

1700

In [14]:
max_sequence_length = 100

# muat model dan text vectorizer config
model = tf.keras.models.load_model("model/model_2p2_90_87.keras")
vector = tf.keras.layers.TextVectorization(
    max_tokens=5000,
    output_mode='int',
    output_sequence_length=max_sequence_length)

# mengubah kolom tanya pada DF menjadi numpy array
training_data = np.array(df['tanya'], dtype=str)

# mengubah input user menjadi numpy array
input_array = np.array([input_users], dtype=str)

# Adaptasikan vector ke Dataframe (training_data)
vector.adapt(training_data)

# prediksi input_array (input dari users)
predictions = model.predict(input_array)

predictions = np.argmax(predictions)
filter_data = df[df['kategori'] == predictions]

# mengubah input_array dan filter_data menjadi vector
input_vector = vector(input_array).numpy()
category_questions = vector(filter_data['tanya']).numpy()

# Hitung cosine similarity antara input pengguna dan semua pertanyaan dalam kategori
cosine_similarities = cosine_similarity(input_vector, category_questions)
most_similar_doc_index = cosine_similarities.argmax()

answer = filter_data.iloc[most_similar_doc_index]['jawab']
print(answer)

1/1 [==============================] - 0s 90ms/step


<p>Halo Fattah,ÃÂ </p>
<p>Makanan pedas biasanya tidak secara langsung membuat sakit kepala atau pusing. Tetapi biasanya komposisi lain pada makanan pedas tersebut, misalnya MSG/mecin yang digunakan sebagai penyedap. Terlalu banyak mengonsumsi MSG dapat berhubungan dengan sakit kepala, walaupun tidak semua mengalaminya. Sebagian orang sensitif pada glutamat pada MSG sehingga dapat merasakan gejala seperti pusing dan mual setelah makan makanan yang banyak mengandung MSG. Makanan yang kadar garamnya terlalu tinggi atau terlalu asin juga bisa memicu sakit kepala.ÃÂ </p>
<p>Makanan pedas bisa membuat asam lambung meningkat dan rasa tidak nyaman di perut, walaupun tidak semua orang mengalaminya. Rasa tidak nyaman ini ada di lambung, di perut sebelah kiri sehingga mungkin hal inilah yang membuat Anda merasa sakit jantung. Rasa tidak nyaman, begah, kembung, mual dan nyeri di perut disebut dengan dispepsia/sakit maag. Dispepsia ini bisa m